# Section 1 - Data Loading

In [3]:
import pandas as pd

df = pd.read_csv("C:/Users/hlias/Desktop/online-retail-cohort-retention/data/raw/online_retail.csv")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


# Section 2 - Data Cleaning

Data cleaning decisions were made to ensure that revenue-based analysis reflects
only valid customer transactions and not cancellations or incomplete records.

In [5]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
df = df.dropna(subset=["CustomerID"])

In [7]:
df = df[~df["InvoiceNo"].astype(str).str.startswith("C")]

In [8]:
df = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)]

In [9]:
df["Revenue"] = df["Quantity"] * df["UnitPrice"]

In [10]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["OrderMonth"] = df["InvoiceDate"].dt.to_period("M")

In [11]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,OrderMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2010-12


In [12]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 397884 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397884 non-null  object        
 1   StockCode    397884 non-null  object        
 2   Description  397884 non-null  object        
 3   Quantity     397884 non-null  int64         
 4   InvoiceDate  397884 non-null  datetime64[ns]
 5   UnitPrice    397884 non-null  float64       
 6   CustomerID   397884 non-null  float64       
 7   Country      397884 non-null  object        
 8   Revenue      397884 non-null  float64       
 9   OrderMonth   397884 non-null  period[M]     
dtypes: datetime64[ns](1), float64(3), int64(1), object(4), period[M](1)
memory usage: 33.4+ MB


,Quantity,InvoiceDate,UnitPrice,CustomerID,Revenue
count,397884.000000,397884,397884.000000,397884.000000,397884.000000
mean,12.988238,2011-07-10 23:41:23.511023360,3.116488,15294.423453,22.397000
min,1.000000,2010-12-01 08:26:00,0.001000,12346.000000,0.001000
25%,2.000000,2011-04-07 11:12:00,1.250000,13969.000000,4.680000
50%,6.000000,2011-07-31 14:39:00,1.950000,15159.000000,11.800000
75%,12.000000,2011-10-20 14:33:00,3.750000,16795.000000,19.800000
max,80995.000000,2011-12-09 12:50:00,8142.750000,18287.000000,168469.600000
std,179.331775,NaN,22.097877,1713.141560,309.071041


# Section 3 - SQL Setup

In [13]:
df["OrderMonth"] = df["OrderMonth"].astype(str)

In [14]:
import sqlite3

conn = sqlite3.connect("../database/retail.db")
df.to_sql("transactions", conn, if_exists="replace", index=False)

397884

# Section 4 - Business KPIs (SQL)

Measures the total revenue generated from completed transactions.

In [15]:
query = """
SELECT
    ROUND(SUM(Revenue), 2) AS total_revenue
FROM transactions;
"""

pd.read_sql_query(query, conn)

,total_revenue
0,8911407.9


Shows how total revenue evolves over time on a monthly basis.

In [16]:
query = """
SELECT
    OrderMonth,
    ROUND(SUM(Revenue), 2) AS monthly_revenue
FROM transactions
GROUP BY OrderMonth
ORDER BY OrderMonth;
"""

pd.read_sql_query(query, conn)

,OrderMonth,monthly_revenue
0,2010-12,572713.89
1,2011-01,569445.04
2,2011-02,447137.35
3,2011-03,595500.76
4,2011-04,469200.36
5,2011-05,678594.56
6,2011-06,661213.69
7,2011-07,600091.01
8,2011-08,645343.90
9,2011-09,952838.38


Counts the number of unique customers who made at least one purchase each month.

In [17]:
query = """
SELECT
    OrderMonth,
    COUNT(DISTINCT CustomerID) AS active_customers
FROM transactions
GROUP BY OrderMonth
ORDER BY OrderMonth;
"""

pd.read_sql_query(query, conn)

,OrderMonth,active_customers
0,2010-12,885
1,2011-01,741
2,2011-02,758
3,2011-03,974
4,2011-04,856
5,2011-05,1056
6,2011-06,991
7,2011-07,949
8,2011-08,935
9,2011-09,1266


Calculates total revenue generated by each customer to identify high-value customers.

In [18]:
query = """
SELECT
    CustomerID,
    ROUND(SUM(Revenue), 2) AS customer_revenue
FROM transactions
GROUP BY CustomerID
ORDER BY customer_revenue DESC;
"""

pd.read_sql_query(query, conn)

,CustomerID,customer_revenue
0,14646.0,280206.02
1,18102.0,259657.30
2,17450.0,194550.79
3,16446.0,168472.50
4,14911.0,143825.06
...,...,...
4333,16878.0,13.30
4334,17956.0,12.75
4335,16454.0,6.90
4336,14792.0,6.20


Segments customers into revenue-based groups to identify the top 20% highest-value customers.

In [19]:
query = """
SELECT
    CustomerID,
    SUM(Revenue) AS revenue,
    NTILE(5) OVER (ORDER BY SUM(Revenue) DESC) AS revenue_bucket
FROM transactions
GROUP BY CustomerID;
"""

pd.read_sql_query(query, conn)


,CustomerID,revenue,revenue_bucket
0,14646.0,280206.02,1
1,18102.0,259657.30,1
2,17450.0,194550.79,1
3,16446.0,168472.50,1
4,14911.0,143825.06,1
...,...,...,...
4333,16878.0,13.30,5
4334,17956.0,12.75,5
4335,16454.0,6.90,5
4336,14792.0,6.20,5


Measures the average revenue per order to understand purchasing behavior.

In [20]:
query = """
SELECT
    ROUND(
        SUM(Revenue) / COUNT(DISTINCT InvoiceNo),
        2
    ) AS avg_order_value
FROM transactions;
"""

pd.read_sql_query(query, conn)

,avg_order_value
0,480.87


# Section 5 - Cohort & Retention Analysis

Analyze customer retention over time by grouping customers into cohorts based on their first purchase month.

A cohort is defined as a group of customers who made their first purchase in the same month.


Create a dedicated cohort table that assigns each transaction to a cohort and calculates the customer lifecycle month.
This table will be reused for all cohort and retention analysis.

In [ ]:
conn.execute("""
CREATE TABLE cohort_data AS
SELECT
    t.CustomerID,
    t.OrderMonth,
    c.CohortMonth,
    (CAST(substr(t.OrderMonth,1,4) AS INT) - CAST(substr(c.CohortMonth,1,4) AS INT)) * 12 +
    (CAST(substr(t.OrderMonth,6,2) AS INT) - CAST(substr(c.CohortMonth,6,2) AS INT)) + 1 AS CohortIndex
FROM transactions t
JOIN (
    SELECT
        CustomerID,
        MIN(OrderMonth) AS CohortMonth
    FROM transactions
    GROUP BY CustomerID
) c
ON t.CustomerID = c.CustomerID;
""")

Validate that cohort assignment and lifecycle calculation are correct.
Each customer should have:
- One CohortMonth
- CohortIndex starting from 1 and increasing over time

In [22]:
query = """
SELECT *
FROM cohort_data
ORDER BY CustomerID, CohortIndex
LIMIT 20;
"""

pd.read_sql_query(query, conn)

,CustomerID,OrderMonth,CohortMonth,CohortIndex
0,12346.0,2011-01,2011-01,1
1,12347.0,2010-12,2010-12,1
2,12347.0,2010-12,2010-12,1
3,12347.0,2010-12,2010-12,1
4,12347.0,2010-12,2010-12,1
5,12347.0,2010-12,2010-12,1
6,12347.0,2010-12,2010-12,1
7,12347.0,2010-12,2010-12,1
8,12347.0,2010-12,2010-12,1
9,12347.0,2010-12,2010-12,1


Calculate the number of unique active customers per cohort and per lifecycle month.

In [23]:
query = """
SELECT
    CohortMonth,
    CohortIndex,
    COUNT(DISTINCT CustomerID) AS active_customers
FROM cohort_data
GROUP BY CohortMonth, CohortIndex
ORDER BY CohortMonth, CohortIndex;
"""

pd.read_sql_query(query, conn)

,CohortMonth,CohortIndex,active_customers
0,2010-12,1,885
1,2010-12,2,324
2,2010-12,3,286
3,2010-12,4,340
4,2010-12,5,321
...,...,...,...
86,2011-10,2,86
87,2011-10,3,41
88,2011-11,1,323
89,2011-11,2,36


Determine the initial size of each cohort based on the number of customers in their first purchase month.

In [24]:
query = """
SELECT
    CohortMonth,
    COUNT(DISTINCT CustomerID) AS cohort_size
FROM cohort_data
WHERE CohortIndex = 1
GROUP BY CohortMonth
ORDER BY CohortMonth;
"""

pd.read_sql_query(query, conn)

,CohortMonth,cohort_size
0,2010-12,885
1,2011-01,417
2,2011-02,380
3,2011-03,452
4,2011-04,300
5,2011-05,284
6,2011-06,242
7,2011-07,188
8,2011-08,169
9,2011-09,299


Calculate retention rate as the percentage of customers who remain active compared to the original cohort size.

In [25]:
query = """
WITH cohort_sizes AS (
    SELECT CohortMonth, COUNT(DISTINCT CustomerID) AS cohort_size
    FROM cohort_data
    WHERE CohortIndex = 1
    GROUP BY CohortMonth
)
SELECT
    r.CohortMonth,
    r.CohortIndex,
    ROUND(1.0 * r.active_customers / c.cohort_size, 2) AS retention_rate
FROM (
    SELECT
        CohortMonth,
        CohortIndex,
        COUNT(DISTINCT CustomerID) AS active_customers
    FROM cohort_data
    GROUP BY CohortMonth, CohortIndex
) r
JOIN cohort_sizes c
ON r.CohortMonth = c.CohortMonth
ORDER BY r.CohortMonth, r.CohortIndex;
"""

pd.read_sql_query(query, conn)

,CohortMonth,CohortIndex,retention_rate
0,2010-12,1,1.00
1,2010-12,2,0.37
2,2010-12,3,0.32
3,2010-12,4,0.38
4,2010-12,5,0.36
...,...,...,...
86,2011-10,2,0.24
87,2011-10,3,0.11
88,2011-11,1,1.00
89,2011-11,2,0.11


# Section 6 - Retention Visualizations

The cohort retention results are visualized in Power BI to enable interactive exploration and stakeholder-friendly reporting.

In [26]:

pd.read_sql_query("SELECT * FROM cohort_data", conn)\
  .to_csv("../data/processed/cohort_data.csv", index=False)